# Trino / Iceberg connector data management demo sample
Copyright (C) 2021 OS-Climate

This sample shows:
* How to create schemas and tables and ingest data from a dataframe via Trino / SQLAlchemy running on an Apache Iceberg data volume
* Simple data set metadata ingestion at the schema and fields level in dedicated Trino data store
* Apache Iceberg ACID transaction and time travel capabilities used for data set versioning


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import pandas as pd
import uuid
import trino
from sqlalchemy.engine import create_engine
from osc_ingest_trino import *

Load Environment Variables

In [3]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Create a simple data frame with list of power plans and capacity for ingestion

In [4]:
# initialise data of lists of powerplants and capacity
data = {'name':['Albas', 'Albi', 'Albias', 'Allaire'],
        'capacity_gwh':[1.8, 6.70448, 2.41, 8.2]}
# Create DataFrame
df = pd.DataFrame(data)
# Add a unique identifier to the data set
uid = str(uuid.uuid4())
df['uuid'] = uid
# Print the output
df = df.convert_dtypes()
df

,name,capacity_gwh,uuid
0,Albas,1.8,fedc4764-a2c7-472b-8c20-5b51b7f85075
1,Albi,6.70448,fedc4764-a2c7-472b-8c20-5b51b7f85075
2,Albias,2.41,fedc4764-a2c7-472b-8c20-5b51b7f85075
3,Allaire,8.2,fedc4764-a2c7-472b-8c20-5b51b7f85075


In [5]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          4 non-null      string 
 1   capacity_gwh  4 non-null      Float64
 2   uuid          4 non-null      string 
dtypes: Float64(1), string(2)
memory usage: 228.0 bytes


Define variables for catalog / schema / table names for ingestion

In [6]:
catalogname = 'osc_datacommons_iceberg_dev'
schemaname = 'wri_dev'
tablename = 'gppd'

Create a connection to Trino database via SQLAlchemy

In [7]:
# telling sqlalchemy about catalog has to be done in the sqlstring url
sqlstring = 'trino://{user}@{host}:{port}/{catalog}'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT'],
    catalog = catalogname
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine))
connection = engine.connect()

connecting with engine Engine(trino://caldeirav@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/osc_datacommons_iceberg_dev)


In [8]:
# Show available schemas to ensure trino connection is set correctly
schema_show_sql = f"""
show schemas in {catalogname}
"""
schema_show = engine.execute(schema_show_sql)
print(schema_show.fetchall())

[('aicoe_osc_demo',), ('company_data',), ('default',), ('defaultschema1',), ('demo',), ('eje',), ('eje_test_iceberg',), ('epa_frs',), ('epa_ghgrp',), ('epacems',), ('epacems_y95_al',), ('essd',), ('gleif',), ('information_schema',), ('ingest_schema',), ('mdt',), ('mdt_dera',), ('mdt_dera1',), ('metastore',), ('metastore_iceberg',), ('osc_corp_data',), ('physical_risk_project',), ('pudl',), ('rmi_utility_transition_hub',), ('sec_dera',), ('sfi_geoasset',), ('team1',), ('team2',), ('testaccessschema1',), ('testdb',), ('urgentem',), ('us_census',), ('wri',), ('wri_demo',), ('wri_gppd',), ('wri_gppd_md',), ('wri_new',), ('wri_test',)]


Create ingestion schema based on source data name, if required

In [9]:
schema_create_sql = f"""
create schema if not exists {catalogname}.{schemaname}
"""
schema_create = engine.execute(schema_create_sql)
print(schema_create.fetchall())

[(True,)]


Create ingestion table if required, by mapping the data frame schema to a SQL schema

In [10]:
schema = create_table_schema_pairs(df)

tabledef = f"""
create table if not exists {catalogname}.{schemaname}.{tablename}(
{schema}
) with (
    format = 'ORC',
    partitioning = ARRAY['uuid']
)
"""
print(tabledef)


create table if not exists osc_datacommons_iceberg_dev.wri_dev.gppd(
    name varchar,
    capacity_gwh double,
    uuid varchar
) with (
    format = 'ORC',
    partitioning = ARRAY['uuid']
)



In [11]:
table_create = engine.execute(tabledef)
print(table_create.fetchall())

[(True,)]


Ingest the power plant data in Append mode

In [12]:
# important specify the schema here, and catalog when you create the db connection earlier
# use 'append' mode since we already created the table
# index = False, unless you declared that as a column when you create the table
# method = 'multi' is important, default will not work
df.to_sql(tablename,
           con=engine,
           schema=schemaname,
           if_exists='append',
           index=False,
           method='multi')

Query Iceberg snapshots for WRI GPPD data set. Snapshots allow having an immutable set of the data at a given time. They are automatically created on every append or removal of data.

In [13]:
snapshot_query=f"""
SELECT committed_at, snapshot_id, parent_id from {catalogname}.{schemaname}."{tablename}$snapshots"
    order by committed_at desc
"""
print(snapshot_query)
snapshots = engine.execute(snapshot_query).fetchall()
snapshots


SELECT committed_at, snapshot_id, parent_id from osc_datacommons_iceberg_dev.wri_dev."gppd$snapshots"
    order by committed_at desc



[('2021-11-29 12:59:39.168 UTC', 6342786149887071998, 6733920372418153154),
 ('2021-11-29 12:59:36.409 UTC', 6733920372418153154, None)]

Create custom meta data for the schema and field-level data

In [14]:
custom_meta_content = {
    'dataset_key': 'gppd',
    'title': 'Global Power Plant Database',
    'description': 'A comprehensive, global, open source database of power plants',
    'version': '1.3.0',
    'release_date': '20210602',
    'fields': [
    {
        'field_name': 'name',
        'definition': 'name of the power plant',
        'type': 'text'
    },
    {
        'field_name': 'capacity_gwh',
        'definition': 'electricity generation in gigawatt-hours',
        'type': 'number'
    }]
}

Convert custom metadata content in json format into Pandas DataFrame

In [15]:
df_meta_fields = pd.json_normalize(custom_meta_content, record_path =['fields'], meta=['dataset_key']).convert_dtypes()
df_meta_fields

,field_name,definition,type,dataset_key
0,name,name of the power plant,text,gppd
1,capacity_gwh,electricity generation in gigawatt-hours,number,gppd


In [16]:
df_meta_fields.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   field_name   2 non-null      string
 1   definition   2 non-null      string
 2   type         2 non-null      string
 3   dataset_key  2 non-null      string
dtypes: string(4)
memory usage: 192.0 bytes


In [17]:
df_meta_table = pd.json_normalize(custom_meta_content, max_level=0)
df_meta_table.drop('fields', inplace=True, axis=1)
df_meta_table['schema'] = schemaname
df_meta_table = df_meta_table.convert_dtypes()
df_meta_table

,dataset_key,title,description,version,release_date,schema
0,gppd,Global Power Plant Database,"A comprehensive, global, open source database ...",1.3.0,20210602,wri_dev


In [18]:
df_meta_table.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dataset_key   1 non-null      string
 1   title         1 non-null      string
 2   description   1 non-null      string
 3   version       1 non-null      string
 4   release_date  1 non-null      string
 5   schema        1 non-null      string
dtypes: string(6)
memory usage: 176.0 bytes


Ingest metadata table for schema (dataset) and field level information

In [19]:
# declare variable names for metadata structure in Trino
meta_schema_name = 'metastore_iceberg'
meta_table_name_dataset = 'meta_tables_iceberg'
meta_table_name_fields = 'meta_fields_iceberg'

In [20]:
# create metastore schema if it does not exist
schema_create_sql = f"""
create schema if not exists {catalogname}.{meta_schema_name}
"""
schema_create = engine.execute(schema_create_sql)
print(schema_create.fetchall())

[(True,)]


In [21]:
# create table for data set level metadata
schema_meta_table = create_table_schema_pairs(df_meta_table)

tabledef = f"""
create table if not exists {catalogname}.{meta_schema_name}.{meta_table_name_dataset}(
{schema_meta_table}
) with (
    format = 'ORC',
    partitioning = ARRAY['dataset_key']
)
"""
print(tabledef)


create table if not exists osc_datacommons_iceberg_dev.metastore_iceberg.meta_tables_iceberg(
    dataset_key varchar,
    title varchar,
    description varchar,
    version varchar,
    release_date varchar,
    schema varchar
) with (
    format = 'ORC',
    partitioning = ARRAY['dataset_key']
)



In [22]:
meta_table_create = engine.execute(tabledef)
print(meta_table_create.fetchall())

[(True,)]


In [23]:
# ingest dataset metadata
df_meta_table.to_sql(meta_table_name_dataset,
           con=engine,
           schema=meta_schema_name,
           if_exists='append',
           index=False,
           method='multi')

In [24]:
# create table for field level metadata
schema_meta_fields = create_table_schema_pairs(df_meta_fields)

tabledef = f"""
create table if not exists {catalogname}.{meta_schema_name}.{meta_table_name_fields}(
{schema_meta_fields}
) with (
    format = 'ORC',
    partitioning = ARRAY['dataset_key']
)
"""
print(tabledef)


create table if not exists osc_datacommons_iceberg_dev.metastore_iceberg.meta_fields_iceberg(
    field_name varchar,
    definition varchar,
    type varchar,
    dataset_key varchar
) with (
    format = 'ORC',
    partitioning = ARRAY['dataset_key']
)



In [25]:
# ingest fields metadata
df_meta_fields.to_sql(meta_table_name_fields,
           con=engine,
           schema=meta_schema_name,
           if_exists='append',
           index=False,
           method='multi')

Update the source data to create a new data set for ingestion

In [26]:
# initialise new data for powerplants and capacity
new_data = {'name':['Albas', 'Albi', 'Albias', 'Allaire'],
        'capacity_gwh':[2.8, 6.90448, 2.20, 7.4]}
# Create DataFrame
new_df = pd.DataFrame(new_data)
# Add a unique identifier to the data set
new_uid = str(uuid.uuid4())
new_df['uuid'] = new_uid
# Print the output
new_df = new_df.convert_dtypes()
new_df

,name,capacity_gwh,uuid
0,Albas,2.8,2b687221-301b-4ce9-86c2-880f65c5151f
1,Albi,6.90448,2b687221-301b-4ce9-86c2-880f65c5151f
2,Albias,2.2,2b687221-301b-4ce9-86c2-880f65c5151f
3,Allaire,7.4,2b687221-301b-4ce9-86c2-880f65c5151f


In [27]:
# Ingest new data set
new_df.to_sql(tablename,
           con=engine,
           schema=schemaname,
           if_exists='append',
           index=False,
           method='multi')

Query data and Iceberg snapshot

In [28]:
dataset_query=f"""
select * from {catalogname}.{schemaname}.{tablename}
"""
print(dataset_query)
pd.read_sql(dataset_query, engine)


select * from osc_datacommons_iceberg_dev.wri_dev.gppd



,name,capacity_gwh,uuid
0,Albas,2.80000,2b687221-301b-4ce9-86c2-880f65c5151f
1,Albi,6.90448,2b687221-301b-4ce9-86c2-880f65c5151f
2,Albias,2.20000,2b687221-301b-4ce9-86c2-880f65c5151f
3,Allaire,7.40000,2b687221-301b-4ce9-86c2-880f65c5151f
4,Albas,1.80000,fedc4764-a2c7-472b-8c20-5b51b7f85075
5,Albi,6.70448,fedc4764-a2c7-472b-8c20-5b51b7f85075
6,Albias,2.41000,fedc4764-a2c7-472b-8c20-5b51b7f85075
7,Allaire,8.20000,fedc4764-a2c7-472b-8c20-5b51b7f85075


In [29]:
# A new data snapshot has been added with the ingestion timestamp, and parent_id as our previous data ingestion
print(snapshot_query)
new_snapshots = engine.execute(snapshot_query).fetchall()
new_snapshots


SELECT committed_at, snapshot_id, parent_id from osc_datacommons_iceberg_dev.wri_dev."gppd$snapshots"
    order by committed_at desc



[('2021-11-29 13:00:13.533 UTC', 3101584315002768302, 6342786149887071998),
 ('2021-11-29 12:59:39.168 UTC', 6342786149887071998, 6733920372418153154),
 ('2021-11-29 12:59:36.409 UTC', 6733920372418153154, None)]

Query only the first data set (Iceberg time machine)

In [30]:
# retrieve the id of the previous snapshot
previous_snapshot = new_snapshots[0][2]
previous_snapshot

6342786149887071998

In [31]:
# query the table state after first snapshot ingestion
past_dataset_query=f"""
select * from {catalogname}.{schemaname}."{tablename}@{previous_snapshot}"
"""
print(past_dataset_query)
pd.read_sql(past_dataset_query, engine)


select * from osc_datacommons_iceberg_dev.wri_dev."gppd@6342786149887071998"



,name,capacity_gwh,uuid
0,Albas,1.80000,fedc4764-a2c7-472b-8c20-5b51b7f85075
1,Albi,6.70448,fedc4764-a2c7-472b-8c20-5b51b7f85075
2,Albias,2.41000,fedc4764-a2c7-472b-8c20-5b51b7f85075
3,Allaire,8.20000,fedc4764-a2c7-472b-8c20-5b51b7f85075


Rollback the second data set

In [32]:
rollback_request=f"""
call {catalogname}.system.rollback_to_snapshot('{schemaname}', '{tablename}', {previous_snapshot})
"""
print(rollback_request)
print(engine.execute(rollback_request).fetchall())


call osc_datacommons_iceberg_dev.system.rollback_to_snapshot('wri_dev', 'gppd', 6342786149887071998)

[(True,)]


In [33]:
# When querying the full table again, only the previous data set is now displayed
pd.read_sql(dataset_query, engine)

,name,capacity_gwh,uuid
0,Albas,1.80000,fedc4764-a2c7-472b-8c20-5b51b7f85075
1,Albi,6.70448,fedc4764-a2c7-472b-8c20-5b51b7f85075
2,Albias,2.41000,fedc4764-a2c7-472b-8c20-5b51b7f85075
3,Allaire,8.20000,fedc4764-a2c7-472b-8c20-5b51b7f85075
